In [ ]:
import random
import os
from PIL import Image, ImageOps
import sys
import tqdm

# import some common libraries
import numpy as np
import os, cv2, random
from detectron2.engine import DefaultPredictor

partial_path = os.getcwd().rsplit("/", 1)[0]
sys.path.insert(0, f"{partial_path}/src/")

from utils import setup_car_detector, detect_car
import config as cf

In [ ]:
# Open images and splices to be applied
images = os.listdir(cf.NO_DAMAGE_IMAGE_PATH)

# Import vehicle detection model
cfg = setup_car_detector()
predictor = DefaultPredictor(cfg)

for index, img in tqdm.tqdm(enumerate(images)):
    image1 = Image.open(os.path.join(cf.NO_DAMAGE_IMAGE_PATH, img))
    image2 = Image.open(os.path.join(cf.NO_DAMAGE_IMAGE_PATH, img))
    width, height = image1.size[0], image1.size[1]
    img_name = img.split(".")[-2]
    
    # Detect the vehicle position
    x,y,w,h = detect_car(os.path.join(cf.NO_DAMAGE_IMAGE_PATH, img), predictor)
    
    # Generate random position inside bbox (conservative choice: box of reduced dimention to be sure of being inside the car)
    x_splice = random.randint(int(x+w/4), int(x+3/4*w))
    y_splice = random.randint(int(y+h/4), int(y+3/4*h)) 
    
    # Pick two random stickers and force them to be different each other
    objects = os.listdir(os.path.join(cf.DAMAGE_STICKERS_PATH))
    
    splice_obj1 = ''
    splice_obj2 = ''
    while splice_obj1==splice_obj2:
        splice_obj1 = objects[random.randint(0, len(objects)-1)]
        splice_obj2 = objects[random.randint(0, len(objects)-1)]
    
    splice_object1 = Image.open(os.path.join(cf.DAMAGE_STICKERS_PATH, splice_obj1))
    splice_object2 = Image.open(os.path.join(cf.DAMAGE_STICKERS_PATH, splice_obj2))

    # Randomly rotate the damages to be spliced
    splice_object1 = splice_object1.rotate(random.randint(-30, 30))
    splice_object2 = splice_object2.rotate(random.randint(-30, 30))
    
    # Apply damage splice resizing
    sizes1 = np.array(image1.size) * (random.randint(5, 10)/100)
    sizes2 = np.array(image1.size) * (random.randint(5, 10)/100)
    
    splice_object1.thumbnail(sizes1.round().astype(int), Image.ANTIALIAS)
    splice_object2.thumbnail(sizes2.round().astype(int), Image.ANTIALIAS)
    
    # Apply change of color and transparency 
    transparency_dice = random.randint(150, 220)
    splice_object1 = splice_object1.convert('RGBA')
    splice_object2 = splice_object2.convert('RGBA')
    datas1 = splice_object1.getdata()
    datas2 = splice_object2.getdata()
    newData1 = []
    newData2 = []
    
    for item in datas1:
        if item[3] != 0:
            newData1.append((item[0], item[1], item[2], transparency_dice))
        else:
            newData1.append(item)
    
    for item in datas2:
        if item[3] != 0:
            newData2.append((item[0], item[1], item[2], transparency_dice))
        else:
            newData2.append(item)
    
    splice_object1.putdata(newData1)
    splice_object2.putdata(newData2)

    # Apply random flip
    flip_coin1 = random.randint(0, 1)
    flip_coin2 = random.randint(0, 1)
    if flip_coin1 == 0:
        splice_object1 = ImageOps.mirror(splice_object1)
    if flip_coin2 == 0:
        splice_object2 = ImageOps.mirror(splice_object2)
    
    # Dilate the damage sticker to fill pixel gaps
    kernel_dilate = np.ones((1, 1), np.uint8)
    
    splice_object1 = np.array(splice_object1)
    splice_object2 = np.array(splice_object2)
    
    splice_object1 = cv2.dilate(splice_object1, kernel_dilate, iterations=2)
    splice_object1 = cv2.blur(splice_object1, (2, 2))
    splice_object2 = cv2.dilate(splice_object2, kernel_dilate, iterations=2)
    splice_object2 = cv2.blur(splice_object2, (2, 2))
    
    splice_object1 = Image.fromarray(splice_object1.astype(np.uint8))
    splice_object2 = Image.fromarray(splice_object2.astype(np.uint8))
    
    # Apply the object
    image1.paste(splice_object1, (x_splice, y_splice), splice_object1.convert('RGBA'))
    image2.paste(splice_object2, (x_splice, y_splice), splice_object2.convert('RGBA'))
    
    # Generate crop around the spliced damage 1
    crop_sizes1 = list(np.array(splice_object1.size))
    crop_sizes2 = list(np.array(splice_object2.size))
    crop_sizes_selected = max([crop_sizes1[0], crop_sizes1[1], crop_sizes2[0], crop_sizes2[1]])
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/2))
    y = max(0, y_splice-(crop_sizes/2))
    image_1_crop_1 = image1.crop((x, y, min(x_splice+crop_sizes/2+crop_sizes1[0], width), min(y_splice+crop_sizes/2+crop_sizes1[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/4))
    y = max(0, y_splice-(crop_sizes/4))
    image_1_crop_2 = image1.crop((x, y, min(x_splice+3/4*crop_sizes+crop_sizes1[0], width), min(y_splice+3/4*crop_sizes+crop_sizes1[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(3/4*crop_sizes))
    y = max(0, y_splice-(crop_sizes/4))
    image_1_crop_3 = image1.crop((x, y, min(x_splice+1/4*crop_sizes+crop_sizes1[0], width), min(y_splice+3/4*crop_sizes+crop_sizes1[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/4))
    y = max(0, y_splice-(3/4*crop_sizes))
    image_1_crop_4 = image1.crop((x, y, min(x_splice+3/4*crop_sizes+crop_sizes1[0], width), min(y_splice+1/4*crop_sizes+crop_sizes1[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(3/4*crop_sizes))
    y = max(0, y_splice-(3/4*crop_sizes))
    image_1_crop_5 = image1.crop((x, y, min(x_splice+1/4*crop_sizes+crop_sizes1[0], width), min(y_splice+1/4*crop_sizes+crop_sizes1[1], height)))
    
    
    # Generate crop around the spliced damage 2
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/2))
    y = max(0, y_splice-(crop_sizes/2))
    image_2_crop_1 = image2.crop((x, y, min(x_splice+crop_sizes/2+crop_sizes2[0], width), min(y_splice+crop_sizes/2+crop_sizes2[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 20)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/4))
    y = max(0, y_splice-(crop_sizes/4))
    image_2_crop_2 = image2.crop((x, y, min(x_splice+3/4*crop_sizes+crop_sizes2[0], width), min(y_splice+3/4*crop_sizes+crop_sizes2[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(3/4*crop_sizes))
    y = max(0, y_splice-(crop_sizes/4))
    image_2_crop_3 = image2.crop((x, y, min(x_splice+1/4*crop_sizes+crop_sizes2[0], width), min(y_splice+3/4*crop_sizes+crop_sizes2[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(crop_sizes/4))
    y = max(0, y_splice-(3/4*crop_sizes))
    image_2_crop_4 = image2.crop((x, y, min(x_splice+3/4*crop_sizes+crop_sizes2[0], width), min(y_splice+1/4*crop_sizes+crop_sizes2[1], height)))
    
    crop_sizes = crop_sizes_selected * (random.randint(0, 10)/100) + crop_sizes_selected
    x = max(0, x_splice-(3/4*crop_sizes))
    y = max(0, y_splice-(3/4*crop_sizes))
    image_2_crop_5 = image2.crop((x, y, min(x_splice+1/4*crop_sizes+crop_sizes2[0], width), min(y_splice+1/4*crop_sizes+crop_sizes2[1], height)))
    
    
    # Generate affine trasformations and crop images accordingly
    m = -0.5
    xshift = 0 
    
    image_1_affine_1 = image_1_crop_1.transform((image_1_crop_1.size[0], image_1_crop_1.size[1]), Image.AFFINE,
                                                (1, m, -xshift if m > 0 else 0, 0, 1, 0), Image.BILINEAR)

    image_1_affine_2 = image_1_crop_1.transform((image_1_crop_1.size[0], image_1_crop_1.size[1]), Image.AFFINE,
                                                (1, -m, -xshift if m > 0 else 0, 0, 1, 0), Image.BILINEAR)
    
    last_row = np.array(image_1_affine_1)[-1][:, 1]
    black_pix = list(last_row).count(0)
    
    image_1_affine_1_crop = image_1_affine_1.crop((black_pix, 0, image_1_affine_1.size[0], image_1_affine_1.size[1]))
    image_1_affine_2_crop = image_1_affine_2.crop((0, 0, image_1_affine_1.size[0]-black_pix, image_1_affine_1.size[1]))
    
    
    # Do the same for other sticker
    image_2_affine_1 = image_2_crop_1.transform((image_2_crop_1.size[0], image_2_crop_1.size[1]), Image.AFFINE,
                                                (1, m, -xshift if m > 0 else 0, 0, 1, 0), Image.BILINEAR)

    image_2_affine_2 = image_2_crop_1.transform((image_2_crop_1.size[0], image_2_crop_1.size[1]), Image.AFFINE,
                                                (1, -m, -xshift if m > 0 else 0, 0, 1, 0), Image.BILINEAR)
    
    last_row = np.array(image_2_affine_1)[-1][:, 1]
    black_pix = list(last_row).count(0)
    
    image_2_affine_1_crop = image_2_affine_1.crop((black_pix, 0, image_2_affine_1.size[0], image_2_affine_1.size[1]))
    image_2_affine_2_crop = image_2_affine_2.crop((0, 0, image_2_affine_2.size[0]-black_pix, image_2_affine_2.size[1]))
    
    # Save the manipulated images
    if not os.path.isdir(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name)):
        os.makedirs(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1'))
        os.makedirs(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2'))
        os.makedirs(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop'))
        os.makedirs(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop'))
    
    image1.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1', img_name+'_'+splice_obj1.split('.')[0]+'.jpg'))
    image2.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2', img_name+'_'+splice_obj2.split('.')[0]+'.jpg'))
    
    image_1_crop_1.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_crop1_'+'.jpg'))
    image_1_crop_2.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_crop2_'+'.jpg'))
    image_1_crop_3.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_crop3_'+'.jpg'))
    image_1_crop_4.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_crop4_'+'.jpg'))
    image_1_crop_5.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_crop5_'+'.jpg'))
    image_1_affine_1_crop.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_affine1_'+'.jpg'))
    image_1_affine_2_crop.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_1_crop', img_name+'_'+splice_obj1.split('.')[0]+'_affine2_'+'.jpg'))
                     
    image_2_crop_1.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj2.split('.')[0]+'_crop1_'+'.jpg'))
    image_2_crop_2.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj2.split('.')[0]+'_crop2_'+'.jpg'))
    image_2_crop_3.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj2.split('.')[0]+'_crop3_'+'.jpg'))
    image_2_crop_4.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj2.split('.')[0]+'_crop4_'+'.jpg'))
    image_2_crop_5.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj2.split('.')[0]+'_crop5_'+'.jpg'))
    image_2_affine_1_crop.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj1.split('.')[0]+'_affine1_'+'.jpg'))
    image_2_affine_2_crop.save(os.path.join(cf.SYNTHETIC_DATASET_PATH, img_name, 'sticker_2_crop', img_name+'_'+splice_obj1.split('.')[0]+'_affine2_'+'.jpg'))
